# References:

https://www.sbert.net/examples/applications/semantic-search/README.html

https://www.sbert.net/docs/pretrained_models.html

https://paperswithcode.com/sota/code-generation-on-humaneval

In this notebook, I loaded datasets containing training, evaluation, and test samples for a question answering task. After extracting questions and contexts from the training and evaluation datasets, I utilized the SentenceTransformer library to load a pre-trained model for encoding text into embeddings. Using this model, I encoded the training and evaluation questions, normalized the embeddings, and employed semantic search to find the most similar question in the training set for each evaluation question. Subsequently, I calculated the accuracy of this model in identifying the correct context for evaluation questions and determined the best-performing model based on accuracy. Furthermore, I authenticated with the Hugging Face Hub and imported necessary libraries to work with a specific pre-trained model called Mistral AI `mistralai/Mistral-7B-v0.1`. After defining its configuration and loading its weights for causal language modeling, I loaded a previously trained and fine-tuned model from a checkpoint directory. To ensure reproducibility, I set a random seed and selected a random test sample from the test dataset. Then, I encoded the test query, found the most similar question in the training set, and retrieved its context. Using this information, I created a prompt and tokenized it for model input. Finally, I generated text based on the model input, disabled gradient calculation during inference, and decoded the generated text to produce a response.

# Load Datasets

In [1]:
from datasets import load_dataset

# Load training dataset
train_dataset = load_dataset('json', data_files='../data/train_CRM_data.json', split='train')

# Load evaluation dataset
eval_dataset = load_dataset('json', data_files='../data/val_CRM_data.json', split='train')

# Load test dataset
test_dataset = load_dataset('json', data_files='../data/test_CRM_data.json', split='train')

# Extract Questions and Contexts

In [2]:
def create_question_context_dict(message):
    """
    Create a dictionary with question as key and context as value

    Args:
        message (list): A list containing dictionaries for question, context, and answer.

    Returns:
        dict: Dictionary of question as key and context as value
    """
    # Extracting question, context, and answer from the message
    question = message[1]['content']
    context = message[0]['content']
    answer = message[2]['content']

    return {question: context}

In [3]:
train_question_context_dict = {}

# Iterate through each example in the training dataset
for sample in train_dataset:
    
    # Extract question and context using create_question_context_dict function
    qc_pair = create_question_context_dict(sample['messages'])
    
    # Merge the extracted pairs into the train_question_context_dict
    train_question_context_dict.update(qc_pair)

eval_question_context_dict = {}

# Iterate through each example in the evaluation dataset
for sample in eval_dataset:
    
    # Extract question and context using create_question_context_dict function
    qc_pair = create_question_context_dict(sample['messages'])
    
    # Merge the extracted pairs into the eval_question_context_dict
    eval_question_context_dict.update(qc_pair)

# Load SentenceTransformer Model, Encode Training Questions, Encode Evaluation Questions

In [4]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load pre-trained SentenceTransformer model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Extracting all questions from train_question_context_dict
train_questions_corpus = list(train_question_context_dict.keys())

# Encode the questions into embeddings
train_corpus_embeddings = embedder.encode(train_questions_corpus, convert_to_tensor=True)

# Move embeddings to GPU if available
train_corpus_embeddings = train_corpus_embeddings.to("cuda")

# Normalize embeddings
train_corpus_embeddings = util.normalize_embeddings(train_corpus_embeddings)

# Query sentences:
eval_questions_queries = list(eval_question_context_dict.keys())

# Encode evaluation questions into embeddings
query_embeddings = embedder.encode(eval_questions_queries, convert_to_tensor=True)

# Move query embeddings to GPU if available
query_embeddings = query_embeddings.to("cuda")

# Normalize query embeddings
query_embeddings = util.normalize_embeddings(query_embeddings)

# Find the most similar question in the training set for each evaluation question
hits = util.semantic_search(query_embeddings, train_corpus_embeddings, score_function=util.dot_score, top_k=1)

# Find Similar Questions

In [5]:
import pandas as pd

output_data = []

# Iterate through each evaluation question and its corresponding hit
for idx, eval_questions_query in enumerate(eval_questions_queries):
    eval_question = eval_questions_query
    
    # Get the best matching train question using the hits
    best_matching_train_question = train_questions_corpus[hits[idx][0]['corpus_id']]
    
    # Check if the correct context is identified by comparing the contexts of the best matching train question and evaluation question
    correct_context_identified = train_question_context_dict[best_matching_train_question] == eval_question_context_dict[eval_questions_query]
    
    # Append the data to the output list
    output_data.append({
        'Eval Question': eval_question,
        'Best Matching Train Question': best_matching_train_question,
        'Correct Context Identified': correct_context_identified
    })

# Creating a DataFrame from the output data
output_df = pd.DataFrame(output_data)

# Displaying the DataFrame
print(output_df)

                                        Eval Question  \
0   What is the correlation between transaction sc...   
1   Most Common Product Category Mentioned in Cust...   
2   Are there any products with a spike in transac...   
3   Could you evaluate the engagement predictions ...   
4   Can you identify any outliers in transaction s...   
5   Are there any products with consistently low t...   
6          Can you identify the top-selling products?   
7   How many transactions have occurred for each c...   
8   Are there any products with a consistent incre...   
9   Are there any trends or patterns in purchasing...   
10  Are there any specific customer segments that ...   
11  How many transactions in our database have a s...   
12  Are there any products that are frequently pur...   
13  Are there any products with a spike in transac...   
14  Are there any trends in the rankings of produc...   
15  Are there any outliers in terms of high-volume...   
16  Are there any external fact

# Calculate Accuracy

In [6]:
# Calculate the total number of evaluations
total_evaluations = len(output_df)

# Calculate the number of correct context identifications
correct_identifications = output_df['Correct Context Identified'].sum()

# Calculate accuracy by dividing the number of correct identifications by the total evaluations
accuracy = correct_identifications / total_evaluations

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.6585365853658537


# Identify Best Model

In [7]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd

def evaluate_model_accuracy(model_name, train_question_context_dict, eval_question_context_dict):
    """
    Evaluate the accuracy of a sentence transformer model on a given set of training and evaluation question-context pairs.

    Args:
        model_name (str): Name of the sentence transformer model to be used for embedding.
        train_question_context_dict (dict): Dictionary mapping training questions to their corresponding contexts.
        eval_question_context_dict (dict): Dictionary mapping evaluation questions to their corresponding contexts.

    Returns:
        float: Accuracy of the model in identifying the correct context for evaluation questions.
    """
    # Load the model
    embedder = SentenceTransformer(model_name)

    # Extracting all questions from train_question_context_dict
    train_questions_corpus = list(train_question_context_dict.keys())
    
    # Encode the training questions into embeddings and move them to GPU if available
    train_corpus_embeddings = embedder.encode(train_questions_corpus, convert_to_tensor=True).to("cuda")
    
    # Normalize the embeddings
    train_corpus_embeddings = util.normalize_embeddings(train_corpus_embeddings)

    # Query sentences:
    eval_questions_queries = list(eval_question_context_dict.keys())
    
    # Encode the evaluation questions into embeddings and move them to GPU if available
    query_embeddings = embedder.encode(eval_questions_queries, convert_to_tensor=True).to("cuda")
    
    # Normalize the embeddings
    query_embeddings = util.normalize_embeddings(query_embeddings)

    # Find the most similar question in the training set for each evaluation question
    hits = util.semantic_search(query_embeddings, train_corpus_embeddings, score_function=util.dot_score, top_k=1)

    output_data = []
    for idx, eval_questions_query in enumerate(eval_questions_queries):
        eval_question = eval_questions_query
        
        # Get the best matching train question using the hits
        best_matching_train_question = train_questions_corpus[hits[idx][0]['corpus_id']]
        
        # Check if the correct context is identified by comparing the contexts of the best matching train question and evaluation question
        correct_context_identified = train_question_context_dict[best_matching_train_question] == eval_question_context_dict[eval_questions_query]

        output_data.append({
            'Eval Question': eval_question,
            'Best Matching Train Question': best_matching_train_question,
            'Correct Context Identified': correct_context_identified
        })

    # Creating a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Calculate the total number of evaluations
    total_evaluations = len(output_df)

    # Calculate the number of correct context identifications
    correct_identifications = output_df['Correct Context Identified'].sum()

    # Calculate accuracy
    accuracy = correct_identifications / total_evaluations

    return accuracy

# List of models to evaluate
model_names = [
    "all-mpnet-base-v2",
    "gtr-t5-xxl", 
    "gtr-t5-xl", 
    "sentence-t5-xxl",
    "gtr-t5-large",
    "all-mpnet-base-v1",
    "multi-qa-mpnet-base-dot-v1",
    "multi-qa-mpnet-base-cos-v1",
    "all-roberta-large-v1",
    "sentence-t5-xl",
    "all-distilroberta-v1",
    "all-MiniLM-L12-v1",
    "all-MiniLM-L12-v2",
    "multi-qa-distilbert-dot-v1",
    "multi-qa-distilbert-cos-v1",
    "gtr-t5-base",
    "sentence-t5-large",
    "all-MiniLM-L6-v2",
    "multi-qa-MiniLM-L6-cos-v1",
    "all-MiniLM-L6-v1",
    "paraphrase-mpnet-base-v2",
    "msmarco-bert-base-dot-v5",
    "multi-qa-MiniLM-L6-dot-v1",
    "sentence-t5-base",
    "msmarco-distilbert-base-tas-b",
    "msmarco-distilbert-dot-v5",
    "paraphrase-distilroberta-base-v2",
    "paraphrase-MiniLM-L12-v2",
    "paraphrase-multilingual-mpnet-base-v2",
    "paraphrase-TinyBERT-L6-v2",
    "paraphrase-MiniLM-L6-v2",
    "paraphrase-albert-small-v2",
    "paraphrase-multilingual-MiniLM-L12-v2",
    "paraphrase-MiniLM-L3-v2",
    "distiluse-base-multilingual-cased-v1",
    "distiluse-base-multilingual-cased-v2",
    "average_word_embeddings_komninos",
    "average_word_embeddings_glove.6B.300d"
]

# Dictionary to store accuracy results
accuracy_results = {}

# Evaluate each model and store accuracy results
for model_name in model_names:
    accuracy = evaluate_model_accuracy(model_name, train_question_context_dict, eval_question_context_dict)
    accuracy_results[model_name] = accuracy

# Print accuracy results
for model_name, accuracy in accuracy_results.items():
    print(f"Model: {model_name}, Accuracy: {accuracy}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/home/jovyan/LLM_CRM_PERSO/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1006: UserWarning: Not enough free disk space to download the file. The expected file size is: 669.90 MB. The target location /home/jovyan/.cache/huggingface/hub only has 496.44 MB free disk space.
  warnings.warn(
/home/jovyan/LLM_CRM_PERSO/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1006: UserWarning: Not enough free disk space to download the file. The expected file size is: 669.90 MB. The target location /home/jovyan/.cache/huggingface/hub/models--sentence-transformers--gtr-t5-large/blobs only has 496.44 MB free disk space.
  warnings.warn(


model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

In [8]:
# Find the model with the highest accuracy
best_model = max(accuracy_results, key=accuracy_results.get)

# Get the accuracy of the best model
best_accuracy = accuracy_results[best_model]

# Print the best model and its accuracy
print(f"The best model based on accuracy is '{best_model}' with an accuracy of {best_accuracy:.2%}.")

The best model based on accuracy is 'all-mpnet-base-v2' with an accuracy of 73.17%.
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


# Authenticate Hugging Face Hub

In [2]:
# Import the notebook_login function from huggingface_hub module
from huggingface_hub import notebook_login

# Call the notebook_login function to authenticate
# Provide an access token from the provided URL - https://huggingface.co/settings/tokens
notebook_login()

# Import Libraries and Load Pretrained Model

In [4]:
# Import necessary libraries
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig
)

# Define Mistral's pretrained model ID
base_model_id = "mistralai/Mistral-7B-v0.1"

# Define BitsAndBytesConfig for Mistral's model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Load the model in 4-bit precision
    bnb_4bit_use_double_quant=True,       # Use double quantization for 4-bit quantization
    bnb_4bit_quant_type="nf4",            # Use nf4 quantization for 4-bit quantization
    bnb_4bit_compute_dtype=torch.bfloat16 # Use bfloat16 for computation with 4-bit quantization
)

# Load Mistral's pretrained model for causal language modeling
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id, 
    quantization_config=bnb_config, # Apply the defined quantization configuration
    device_map="auto"               # Automatically select the device for model inference
)

# Load Mistral's tokenizer
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id, 
    add_bos_token=True,     # Add beginning-of-sequence token
    trust_remote_code=True  # Trust remote code for tokenization
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Load Fine-Tuned Model

In [5]:
from peft import PeftModel

# Load the weights from the checkpoint directory
ft_model = PeftModel.from_pretrained(base_model, "../models/01-finetune-mistral/checkpoint-150")

# Retrieve context for a random test sample question and prepare prompt

In [6]:
import random

# Set the seed for reproducibility
random.seed(42)

# Take a random index
random_index = random.randint(0, len(test_dataset) - 1)

# Take the random sample
random_test_sample = test_dataset[random_index]

embedder = SentenceTransformer("all-mpnet-base-v2")

# Extracting all questions from train_question_context_dict
train_questions_corpus = list(train_question_context_dict.keys())

# Encode the training questions into embeddings and move them to GPU if available
train_corpus_embeddings = embedder.encode(train_questions_corpus, convert_to_tensor=True).to("cuda")

# Normalize the embeddings
train_corpus_embeddings = util.normalize_embeddings(train_corpus_embeddings)

# Query sentence:
test_query = [random_test_sample['messages'][1]['content']]

# Encode the test query into embeddings and move them to GPU if available
query_embedding = embedder.encode(test_query, convert_to_tensor=True).to("cuda")

# Normalize the embeddings
query_embedding = util.normalize_embeddings(query_embedding)

# Find the most similar question in the training set for the test query
hits = util.semantic_search(query_embedding, train_corpus_embeddings, score_function=util.dot_score, top_k=1)

# Retrieve the context corresponding to the most similar question
retrieved_context = train_question_context_dict[train_questions_corpus[hits[0][0]['corpus_id']]]

# Create a prompt with the test query and the retrieved context
prompt = f"""
### Question: {test_query[0]}

### Context: {retrieved_context}
### Answer:
""".strip()

# Init a tokenizer that doesn't add padding or eos token
test_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

# Tokenize the test prompt and prepare model input for inference
model_input = test_tokenizer(
    prompt, 
    return_tensors="pt"
).to("cuda")  # Move tensors to GPU if available

# Set the fine-tuned model to evaluation mode
ft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

# Model Inferencing

In [7]:
# Disable gradient calculation during inference
with torch.no_grad():
    # Generate text based on the model input
    generated_tokens = ft_model.generate(
        **model_input,                            # Pass model input
        max_new_tokens=1024,                      # Maximum number of new tokens to generate
        repetition_penalty=1.15,                  # Repetition penalty to avoid repetition
        pad_token_id=eval_tokenizer.eos_token_id  # Set pad token ID
    )[0]                                          # Get the first generated sequence

    # Decode the generated tokens into text, skipping special tokens
    generated_text = eval_tokenizer.decode(
        generated_tokens,         # Generated tokens
        skip_special_tokens=True  # Skip special tokens like padding and eos
    )

    # Print the generated text
    print(generated_text)

2024-05-06 13:05:56.766195: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-06 13:05:56.824131: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 13:05:57.652431: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Question: What is the average transaction score for each product category?

### Context: 
You are a Python function generator. Users will ask you questions in English, 
and you will produce a Python function as answer to the question based on the provided CONTEXT.

CONTEXT:
Pandas DataFrame df containing transaction data with columns order_id, user_id, item_id, timestamp, score.
order_id takes string datatype and identifies the order.
user_id takes string datatype and identifies the customer.
item_id takes string datatype that identifies the product.
timestamp takes timestamp datatype and represents the datetimestamp of transaction.
score takes float datatype and represents the score of the transaction.
Note that a customer can make multiple transactions for a product but the customer product pair will be just one entry for each order.
Pandas DataFrame customer_dfcontaining customer data with columns user_id, customer_city.
user_id takes string datatype and identifies the customer.

Retrieval Augmented Generation (RAG) indeed leverages semantic search to enhance text generation by retrieving relevant context from a large database. This approach ensures that generated text is more accurate and contextually appropriate. Using powerful VectorDBs can certainly scale up the performance of semantic search, making it more efficient and effective in handling large volumes of data. This combination of techniques holds a lot of promise for improving natural language generation tasks across various domains.

# Extracting Answer Section from Generated Text

In [8]:
# Find the start position of the answer section
start_marker = "### Answer:"
start_index = generated_text.find(start_marker) + len(start_marker)

# If the start marker is found
if start_index != -1:
    # Extract the answer
    answer = generated_text[start_index:].strip()
    # Check if there is an explanation part
    if "### Explanation:" in answer:
        # Find the end position of the answer section
        end_marker = "### Explanation:"
        end_index = answer.find(end_marker)
        # Extract only the answer part
        answer = answer[:end_index].strip()
else:
    answer = "No answer found."

answer = answer.replace("\\\\", "\\").strip().replace("\\n", "\n").rstrip('\\').replace("\\'", "'").strip()
print(answer)

def avg_transaction_score_per_category(df, customer_df, product_df):
    merged_df = pd.merge(df, customer_df, on='user_id')
    merged_df = pd.merge(merged_df, product_df, on='item_id')
    category_scores = merged_df.groupby('product_category')['score'].mean()
    return category_scores


# Parsing Function Definitions

In [9]:
import ast
import astor

# Function to parse function definition string
def parse_function_definition(definition_string):
    """
    This function parses a given function definition string and extracts important information such as function name, arguments, and body statements. It then constructs a complete function definition string by adding necessary imports and the original function definition.

    Parameters:
    - `definition_string`: The string representing the function definition.

    Returns:
    - `function_name`: Name of the parsed function.
    - `arguments`: List of arguments of the parsed function.
    - `body`: Body statements of the parsed function.
    - `function_definition`: Complete function definition string including necessary imports and the original function definition.
    """
    # Parse the definition string
    parsed = ast.parse(definition_string)
    
    # Initialize variables
    function_name = None
    arguments = []
    body = []
    
    # Iterate over parsed body
    for node in parsed.body:
        # Check if node is a function definition
        if isinstance(node, ast.FunctionDef):
            # Get function name
            function_name = node.name
            # Get function arguments
            arguments = [arg.arg for arg in node.args.args]
            
            # Iterate over function body statements
            for stmt in node.body:
                # Convert AST node to source code and append to body list
                body.append(astor.to_source(stmt).strip())
    
    # Construct complete function definition string
    function_definition = (
        "\n"
        # Import necessary modules
        f"import pandas as pd\n"
        f"import numpy as np\n"
        f"from numpy.linalg import LinAlgError\n"
        f"from datetime import datetime, timedelta\n"
        f"from collections import defaultdict, Counter\n"
        f"from itertools import combinations\n"
        f"from scipy.sparse import csr_matrix\n"
        f"from scipy.stats import zscore\n"
        f"from sklearn.preprocessing import StandardScaler, LabelEncoder\n"
        f"from sklearn.cluster import KMeans\n"
        f"from sklearn.model_selection import train_test_split\n"
        f"from sklearn.linear_model import LogisticRegression, LinearRegression\n"
        f"from sklearn.metrics.pairwise import cosine_similarity\n"
        f"from sklearn.feature_extraction.text import TfidfVectorizer\n"
        f"from sklearn.decomposition import TruncatedSVD\n"
        f"from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score\n"
        f"from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor\n"
        f"from mlxtend.frequent_patterns import apriori\n"
        f"from mlxtend.frequent_patterns import association_rules\n"
        f"from surprise import Reader, Dataset\n"
        f"from surprise.prediction_algorithms import SVD\n"
        f"from matplotlib import pyplot as plt\n"
        f"from statsmodels.tsa.statespace.sarimax import SARIMAX\n"
        f"from statsmodels.tools.sm_exceptions import ConvergenceWarning\n"
        f"from statsmodels.tsa.seasonal import seasonal_decompose\n"
        f"import warnings\n"
        # Append original function definition
        f"{definition_string}"
    )
    return function_name, arguments, '\n'.join(body), function_definition

# Parse the function definition and retrieve function name, arguments, and complete function definition
function_name, arguments, _, function_definition = parse_function_definition(answer)

# Print function name, arguments, and complete function definition
print(function_name)
print(arguments)
print(function_definition)

avg_transaction_score_per_category
['df', 'customer_df', 'product_df']

import pandas as pd
import numpy as np
from numpy.linalg import LinAlgError
from datetime import datetime, timedelta
from collections import defaultdict, Counter
from itertools import combinations
from scipy.sparse import csr_matrix
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sur

# Executing Parsed Function Definition with Sample Data

In [10]:
import pandas as pd
import random
from faker import Faker
import inspect

# Initialize Faker for generating fake data
fake = Faker()

# List of famous city names
european_cities = [
    'London', 'Paris'
]

# Generate sample data for orders
orders_data = []
for i in range(1, 16):
    order_id = f'order{i}'
    user_id = f'user{random.randint(1, 5)}'
    item_id = f'item{random.randint(1, 3)}'
    timestamp = fake.date_time_between(start_date='-5d', end_date='now')
    score = round(random.uniform(0.5, 1.0), 2)
    orders_data.append({'order_id': order_id, 'user_id': user_id, 'item_id': item_id, 'timestamp': timestamp, 'score': score})
orig_df = pd.DataFrame(orders_data)

# Generate sample data for customers
customers_data = []
for i in range(1, 6):
    user_id = f'user{i}'
    customer_city = random.choice(european_cities)
    customers_data.append({'user_id': user_id, 'customer_city': customer_city})
orig_customer_df = pd.DataFrame(customers_data).drop_duplicates(subset=['user_id'])

# Generate sample data for products
products_data = []
for i in range(1, 6):
    item_id = f'item{i}'
    product_category = fake.word()
    products_data.append({'item_id': item_id, 'product_category': product_category})
orig_product_df = pd.DataFrame(products_data).drop_duplicates(subset=['item_id'])

# Function to copy original data frames
def data_copy(orig_df, orig_customer_df, orig_product_df):
    """
    This function creates deep copies of the original data frames to prevent modification of the original data.

    Parameters:
    - `orig_df`: Original DataFrame containing transaction data.
    - `orig_customer_df`: Original DataFrame containing customer data.
    - `orig_product_df`: Original DataFrame containing product data.

    Returns:
    - `df`: Deep copy of the original transaction DataFrame.
    - `customer_df`: Deep copy of the original customer DataFrame.
    - `product_df`: Deep copy of the original product DataFrame.
    """
    df = orig_df.copy(deep=True)
    customer_df = orig_customer_df.copy(deep=True)
    product_df = orig_product_df.copy(deep=True)
    return df, customer_df, product_df

# Copy original data frames
df, customer_df, product_df = data_copy(orig_df, orig_customer_df, orig_product_df)

# Displaying sample data
print("Orders Data:")
print(df)
print("\nCustomer Data:")
print(customer_df)
print("\nProduct Data:")
print(product_df)

# Execute the parsed function definition
globals_ = {'df': df, 'customer_df': customer_df, 'product_df': product_df}
exec(function_definition, globals_)
_parsed_function = globals_[function_name]

# Get function signature and default parameter values
_signature = inspect.signature(_parsed_function)
_parameters_with_defaults = [(param.name, param.default) for param in _signature.parameters.values() if param.default != inspect.Parameter.empty]
_default_values = dict(_parameters_with_defaults)

# Prepare arguments for function execution
_args = tuple(_default_values[arg] if arg in _default_values else globals()[arg] for arg in arguments)

# Call the parsed function with appropriate arguments
result = _parsed_function(*_args)

# Print the result
print(f"\nAnswer to user query:\n{result}")

Orders Data:
   order_id user_id item_id                  timestamp  score
0    order1   user1   item1 2024-05-03 09:35:07.574568   0.87
1    order2   user2   item1 2024-05-05 18:32:24.651137   0.57
2    order3   user1   item3 2024-05-04 08:18:30.474318   0.87
3    order4   user5   item1 2024-05-06 02:07:19.433402   0.80
4    order5   user1   item1 2024-05-02 07:25:38.185993   0.55
5    order6   user2   item3 2024-05-03 08:24:52.126336   0.80
6    order7   user5   item1 2024-05-06 05:22:24.670369   0.86
7    order8   user5   item2 2024-05-04 20:40:11.943480   0.61
8    order9   user5   item2 2024-05-05 01:39:06.918721   0.90
9   order10   user1   item1 2024-05-04 03:06:28.890031   0.85
10  order11   user3   item2 2024-05-03 00:16:38.741952   0.58
11  order12   user3   item1 2024-05-03 16:46:54.816524   0.55
12  order13   user1   item2 2024-05-05 02:16:28.347155   0.92
13  order14   user5   item2 2024-05-02 08:43:39.155055   0.90
14  order15   user4   item3 2024-05-06 09:07:40.112240   

# Results retrieval for user queries in test dataset and measuring performance

In [11]:
def encode_and_move_embeddings_to_gpu(query, embedder):
    """
    Encodes a query and moves the embeddings to GPU.

    Args:
        query (str): The input query to encode.
        embedder: The embedding model to use for encoding.

    Returns:
        torch.Tensor: The encoded query embeddings on GPU.
    """
    start_time = time.time()
    query_embedding = embedder.encode(query, convert_to_tensor=True).to("cuda")
    logging.info(f"\nTime for encoding and moving embeddings to GPU: {time.time() - start_time} seconds")
    return query_embedding

def normalize_embeddings(embeddings, util):
    """
    Normalizes embeddings.

    Args:
        embeddings (torch.Tensor): The embeddings to normalize.
        util: The utility object for normalization.

    Returns:
        torch.Tensor: The normalized embeddings.
    """
    start_time = time.time()
    normalized_embeddings = util.normalize_embeddings(embeddings)
    logging.info(f"\nTime for normalizing embeddings: {time.time() - start_time} seconds")
    return normalized_embeddings

def semantic_search(query_embedding, train_corpus_embeddings, util):
    """
    Performs semantic search using query embedding and train corpus embeddings.

    Args:
        query_embedding (torch.Tensor): The embedding of the query.
        train_corpus_embeddings (torch.Tensor): The embeddings of the training corpus.
        util: The utility object for semantic search.

    Returns:
        list: List of hits from the semantic search.
    """
    start_time = time.time()
    hits = util.semantic_search(query_embedding, train_corpus_embeddings, score_function=util.dot_score, top_k=1)
    logging.info(f"\nTime for semantic search: {time.time() - start_time} seconds")
    return hits

def retrieve_context(hits, train_question_context_dict, train_questions_corpus):
    """
    Retrieves context based on hits from semantic search.

    Args:
        hits (list): List of hits from semantic search.
        train_question_context_dict (dict): Dictionary mapping questions to their contexts.
        train_questions_corpus (list): List of questions in the training corpus.

    Returns:
        str: Retrieved context.
    """
    start_time = time.time()
    retrieved_context = train_question_context_dict[train_questions_corpus[hits[0][0]['corpus_id']]]
    logging.info(f"\nTime for retrieving context: {time.time() - start_time} seconds")
    return retrieved_context

def create_prompt(test_query, retrieved_context):
    """
    Creates a prompt for generating the answer.

    Args:
        test_query (str): The test query.
        retrieved_context (str): The retrieved context.

    Returns:
        str: The generated prompt.
    """
    prompt = f"""
    ### Question: {test_query}

    ### Context: {retrieved_context}
    USE ONLY DEFINED DATAFRAMES AND INCLUDE THESE AS ARGUMENTS.
    NO NEED TO CONSIDER ANY EXTERNAL DATA SOURCES, VARIABLES OR DATAFRAMES APART FROM THE ONES ALREADY AVAILABLE IN THE GLOBALS.
    THE GENERATED ANSWER SHOULD BE A SINGLE PYTHON FUNCTION WITH A `def` AND `return` ONLY.
    THE GENERATED CODE SHOULD BE COMPLETE IN ITSELF, SYNTACTICALLY CORRECT AND BE PARSED SUCCESSFULLY.
    NO NEED TO CALL THE FUNCTION OR PRINT THE FUNCTION CALL. NO NEED TO INCLUDE MAIN CALL. JUST PROVIDING THE PYTHON FUNCTION IS SUFFICIENT.
    
    ### Answer:
    """.strip()
    return prompt

def tokenize_prompt(prompt, test_tokenizer):
    """
    Tokenizes the prompt.

    Args:
        prompt (str): The prompt to tokenize.
        test_tokenizer: The tokenizer to use.

    Returns:
        dict: Model input with tokenized prompt.
    """
    start_time = time.time()
    model_input = test_tokenizer(prompt, return_tensors="pt").to("cuda")
    logging.info(f"\nTime for tokenizing prompt: {time.time() - start_time} seconds")
    return model_input

def generate_text(model_input, ft_model, eval_tokenizer):
    """
    Generates text based on model input.

    Args:
        model_input (dict): Model input with tokenized prompt.
        ft_model: The fine-tuned model for text generation.
        eval_tokenizer: The tokenizer for evaluation.

    Returns:
        torch.Tensor: Generated tokens.
    """
    start_time = time.time()
    with torch.no_grad():
        generated_tokens = ft_model.generate(
            **model_input,
            max_new_tokens=1024,
            repetition_penalty=1.15,
            pad_token_id=eval_tokenizer.eos_token_id
        )[0]
    logging.info(f"\nTime for generating text: {time.time() - start_time} seconds")
    return generated_tokens

def decode_generated_text(generated_tokens, eval_tokenizer):
    """
    Decodes generated text tokens.

    Args:
        generated_tokens (torch.Tensor): Generated tokens.
        eval_tokenizer: The tokenizer for evaluation.

    Returns:
        str: Decoded generated text.
    """
    start_time = time.time()
    generated_text = eval_tokenizer.decode(
        generated_tokens,
        skip_special_tokens=True
    )
    logging.info(f"\nTime for decoding generated text: {time.time() - start_time} seconds")
    return generated_text

def process_answer(generated_text):
    """
    Processes the generated text to extract the answer.

    Args:
        generated_text (str): The generated text.

    Returns:
        str: The processed answer.
    """
    start_time = time.time()
    start_marker = "### Answer:"
    start_index = generated_text.find(start_marker) + len(start_marker)

    if start_index == -1:
        return "No answer found."

    answer = generated_text[start_index:].strip()

    end_marker = None
    if "### Explanation:" in answer:
        end_marker = "### Explanation:"
    elif "### " in answer:
        end_marker = "### "

    if end_marker:
        end_index = answer.find(end_marker)
        if end_index != -1:
            answer = answer[:end_index].strip()

    end_of_function_index = answer.find("\r\n")
    if end_of_function_index != -1:
        answer = answer[:end_of_function_index].strip()

    end_of_function_index = answer.find("if __name__ ==")
    if end_of_function_index != -1:
        answer = answer[:end_of_function_index].strip()

    answer = answer.replace("\\\\", "\\").strip().replace("\\n", "\n").rstrip('\\').replace("\\'", "'").strip()
    logging.info(f"\nTime for processing answer string: {time.time() - start_time} seconds")
    return answer

def print_assistant_message(function_definition):
    """
    Prints a message from the assistant.

    Args:
        function_definition (str): The definition of the function.
    """
    print(f"""
    Assistant:
    {function_definition}
    """)

def execute_function(parsed_function, globals_, arguments):
    """
    Executes a parsed function.

    Args:
        parsed_function (str): The name of the function to execute.
        globals_ (dict): Global namespace dictionary.
        arguments (tuple): Arguments to pass to the function.

    Returns:
        any: Result of the executed function.
    """
    start_time = time.time()
    _parsed_function = globals_[parsed_function]
    _signature = inspect.signature(_parsed_function)
    _parameters_with_defaults = [(param.name, param.default) for param in _signature.parameters.values() if param.default != inspect.Parameter.empty]
    _default_values = dict(_parameters_with_defaults)
    _args = tuple(_default_values[arg] if arg in _default_values else globals()[arg] for arg in arguments)
    result = _parsed_function(*_args)
    logging.info(f"\nTime for executing function: {time.time() - start_time} seconds")
    return result

In [ ]:
import logging
import time
import inspect

# Configure logging
logging.basicConfig(level=logging.INFO)

# Counter for tracking errors
cnt = 0
cnt_2 = 0
cnt_3 = 0

# Loop through test dataset
for idx, _ in enumerate(test_dataset):
    loop_start_time = time.time()
    
    # Counter for code parsing status
    code_parse_ind = 0
    
    # Retrieve test sample
    test_sample = test_dataset[idx]

    # Query sentence:
    test_query = [test_sample['messages'][1]['content']]
    print(f"\nUser Query {idx+1}: {test_query[0]}")
    
    # Encode the test query into embeddings and move them to GPU if available
    query_embedding = encode_and_move_embeddings_to_gpu(test_query, embedder)

    # Normalize the embeddings
    query_embedding = normalize_embeddings(query_embedding, util)

    # Find the most similar question in the training set for the test query
    hits = semantic_search(query_embedding, train_corpus_embeddings, util)

    # Retrieve the context corresponding to the most similar question
    retrieved_context = retrieve_context(hits, train_question_context_dict, train_questions_corpus)

    # Create a prompt with the test query and the retrieved context
    prompt = create_prompt(test_query[0], retrieved_context)

    # Tokenize the test prompt and prepare model input for inference
    model_input = tokenize_prompt(prompt, test_tokenizer)

    # Generate text based on the model input
    generated_tokens = generate_text(model_input, ft_model, eval_tokenizer)

    # Decode the generated tokens into text, skipping special tokens
    generated_text = decode_generated_text(generated_tokens, eval_tokenizer)
    
    # Process the generated text to extract the answer
    answer = process_answer(generated_text)
    
    try:
        
        # Parse the function definition and retrieve function name, arguments, and complete function definition
        start_time = time.time()
        function_name, arguments, body, function_definition = parse_function_definition(answer)
        logging.info(f"\nTime for parsing function definition: {time.time() - start_time} seconds")
        start_time = time.time()
        
        print_assistant_message(function_definition)
        
        code_parse_ind = 1
        
        globals_ = {'df': df, 'customer_df': customer_df, 'product_df': product_df}
        
        exec(function_definition, globals_)
        
        # Execute the parsed function definition
        result = execute_function(function_name, globals_, arguments)
        
        # Print the result
        print(f"Result:\n{result}")

    except Exception as err_1:
        if code_parse_ind == 0:
            print(f"\nGenerated Answer:\n{answer}")
        cnt += 1
        print(f"ERROR: {err_1} | counter 1: {cnt}")
        
        ##### 2nd pass #####
        
        error_prompt = f"""
        ### Question: {test_query}. Do not use {err_1} in your response.
        
        ### Context: {retrieved_context}
        
        ### Answer:
        """.strip()
        model_input = tokenize_prompt(error_prompt, test_tokenizer)
        generated_tokens = generate_text(model_input, ft_model, eval_tokenizer)
        generated_text = decode_generated_text(generated_tokens, eval_tokenizer)
        answer = process_answer(generated_text)
        try:
            function_name, arguments, body, function_definition = parse_function_definition(answer)
            print_assistant_message(function_definition)
            code_parse_ind = 1
            globals_ = {'df': df, 'customer_df': customer_df, 'product_df': product_df}
            exec(function_definition, globals_)
            result = execute_function(function_name, globals_, arguments)
            print(f"Result:\n{result}")
        except Exception as err_2:
            if code_parse_ind == 0:
                print(f"\nGenerated Answer:\n{answer}")
            cnt_2 += 1
            print(f"ERROR: {err_2} | counter 2: {cnt_2}")
            
            ##### 3rd pass #####
        
            error_prompt = f"""
            ### Question: {test_query}. Do not use {err_1} and {err_2} in your response.

            ### Context: {retrieved_context}

            ### Answer:
            """.strip()
            model_input = tokenize_prompt(error_prompt, test_tokenizer)
            generated_tokens = generate_text(model_input, ft_model, eval_tokenizer)
            generated_text = decode_generated_text(generated_tokens, eval_tokenizer)
            answer = process_answer(generated_text)
            try:
                function_name, arguments, body, function_definition = parse_function_definition(answer)
                print_assistant_message(function_definition)
                code_parse_ind = 1
                globals_ = {'df': df, 'customer_df': customer_df, 'product_df': product_df}
                exec(function_definition, globals_)
                result = execute_function(function_name, globals_, arguments)
                print(f"Result:\n{result}")
            except Exception as err_3:
                if code_parse_ind == 0:
                    print(f"\nGenerated Answer:\n{answer}")
                cnt_3 += 1
                print(f"ERROR: {err_3} | counter 3: {cnt_3}")
            
        #####

    logging.info(f"\nTotal loop execution time: {time.time() - loop_start_time} seconds")

In [20]:
print(f"\nTotal errors (Pass 1): {cnt}")
print(f"\nTotal errors (Pass 2): {cnt_2}")
print(f"\nTotal errors (Pass 3): {cnt_3}")
print(f"\nTotal test samples: {len(test_dataset)}")
print(f"\n1st pass Accuracy: {(1-(cnt/len(test_dataset)))*100}%")
print(f"\n2nd pass Accuracy: {(1-(cnt_2/len(test_dataset)))*100}%")
print(f"\n3rd pass Accuracy: {(1-(cnt_3/len(test_dataset)))*100}%")


Total errors (Pass 1): 10

Total errors (Pass 2): 7

Total errors (Pass 3): 5

Total test samples: 42

1st pass Accuracy: 76.19047619047619%

2nd pass Accuracy: 83.33333333333334%

3rd pass Accuracy: 88.09523809523809%


We observe an improvement in accuracy with each subsequent pass. This improvement suggests that the error handling strategy implemented, which includes generating a new response excluding previously encountered errors, is effective in enhancing the overall accuracy of the system. However, despite the improvement, there are still errors encountered during the testing phase. This indicates that there might be inherent limitations in the model. Further analysis of the types of errors encountered and their root causes could provide insights into areas for improvement. If we train the model with these erroneous observations as additional context, the model can provide better accuracy in zero-shot. Additionally, it's important to consider the complexity and diversity of the test dataset. This leads to the insight that the dataset contains a wide range of scenarios and edge cases, achieving a higher accuracy might require more sophisticated error handling mechanisms or model enhancements. While the iterative error handling strategy has led to an improvement in accuracy, continued refinement and evaluation are necessary to further enhance the performance of the system.